In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from functools import reduce

In [2]:
spark = SparkSession.builder.appName("Check_FIPEZAP").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/02 11:34:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:

df_fipezap = spark.read.option("delimiter", ",").option("header", True).csv("dados_fipezap.csv")\
.withColumn("DT_REG_IC", F.lit(F.to_date(F.col("DT_REG_IC"), "yyyy-MM-dd")))\
.withColumn("VL_IC", F.round(F.col("VL_IC"), 2))\
.withColumn("VL_VRC_IC", F.round(F.col("VL_VRC_IC"), 2))\
.withColumn("DT_REF", F.lit(F.to_date(F.col("DT_REF"), "yyyy-MM-dd")))

df_fipezap = df_fipezap\
.withColumn("VL_VRC_IC", F.when(F.col("VL_VRC_IC").isNotNull(), F.col("VL_VRC_IC")).otherwise(0.00))\
.withColumn("VL_IC", F.col("VL_IC").cast("float"))\
.withColumn("VL_VRC_IC", F.col("VL_VRC_IC").cast("float"))

df_fipezap.write.parquet("indices_mercado_externo.parquet", mode="overwrite")

df_fipezap.orderBy("DT_REG_IC", ascending=False).show(truncate=False)

+----------+-------+------+---------+----------+
|DT_REG_IC |NM_IC  |VL_IC |VL_VRC_IC|DT_REF    |
+----------+-------+------+---------+----------+
|2024-04-01|FIPEZAP|160.72|0.66     |2024-06-02|
|2024-03-01|FIPEZAP|159.66|0.64     |2024-06-02|
|2024-02-01|FIPEZAP|158.65|0.49     |2024-06-02|
|2024-01-01|FIPEZAP|157.87|0.36     |2024-06-02|
|2023-12-01|FIPEZAP|157.31|0.29     |2024-06-02|
|2023-11-01|FIPEZAP|156.85|0.37     |2024-06-02|
|2023-10-01|FIPEZAP|156.27|0.54     |2024-06-02|
|2023-09-01|FIPEZAP|155.44|0.45     |2024-06-02|
|2023-08-01|FIPEZAP|154.74|0.44     |2024-06-02|
|2023-07-01|FIPEZAP|154.07|0.41     |2024-06-02|
|2023-06-01|FIPEZAP|153.44|0.51     |2024-06-02|
|2023-05-01|FIPEZAP|152.65|0.46     |2024-06-02|
|2023-04-01|FIPEZAP|151.96|0.45     |2024-06-02|
|2023-03-01|FIPEZAP|151.29|0.42     |2024-06-02|
|2023-02-01|FIPEZAP|150.65|0.38     |2024-06-02|
|2023-01-01|FIPEZAP|150.08|0.3      |2024-06-02|
|2022-12-01|FIPEZAP|149.64|0.3      |2024-06-02|
|2022-11-01|FIPEZAP|

In [9]:
df_fipezap.printSchema()

root
 |-- DT_REG_IC: date (nullable = true)
 |-- NM_IC: string (nullable = true)
 |-- VL_IC: float (nullable = true)
 |-- VL_VRC_IC: float (nullable = true)
 |-- DT_REF: date (nullable = true)



In [10]:
df_csv = df_fipezap
df_base = spark.read.parquet("indices_mercado_externo.parquet")

In [11]:
df_base.printSchema()

root
 |-- DT_REG_IC: date (nullable = true)
 |-- NM_IC: string (nullable = true)
 |-- VL_IC: float (nullable = true)
 |-- VL_VRC_IC: float (nullable = true)
 |-- DT_REF: date (nullable = true)

